In [1]:
!pip install xgboost
!pip install tensorflow
!pip install tensorboardX

     |████████████████████████████████| 192.9 MB 129 kB/s             
     |████████████████████████████████| 497.6 MB 19 kB/s              
     |████████████████████████████████| 462 kB 9.6 MB/s            
     |████████████████████████████████| 77 kB 4.5 MB/s             
     |████████████████████████████████| 14.5 MB 8.6 MB/s            
     |████████████████████████████████| 1.4 MB 15.8 MB/s            
     |████████████████████████████████| 65 kB 2.6 MB/s             
     |████████████████████████████████| 57 kB 4.3 MB/s             
     |████████████████████████████████| 2.1 MB 12.1 MB/s            
     |████████████████████████████████| 5.8 MB 12.8 MB/s            
     |████████████████████████████████| 126 kB 15.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 42 kB 1.7 MB/s             
     |████████████████████████████████| 4.4 MB 7.4 MB/s            
     |████████████████████████████████| 4.9 MB 5.8 MB/s         

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
import xgboost as xgb
from tensorboardX import SummaryWriter
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [4]:
import tensorflow as tf
tf.__version__

2022-05-14 06:20:03.354510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-14 06:20:03.354590: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


'2.8.0'

In [5]:
import tensorboardX
tensorboardX.__version__

'2.5'

In [6]:
xgb.__version__

'1.6.1'

In [7]:
class TensorBoardCallback(xgb.callback.TrainingCallback):
    '''
    Callback for add loss to summary.
    '''
    def __init__(self):
        self.writer = SummaryWriter()

    def _get_key(self, data, metric):
        return f'{data}-{metric}'

    def after_iteration(self, model, epoch, evals_log):
        '''Add scalar loss data to summary.'''
        self.writer.add_scalar("train_loss", evals_log["train"]["rmse"][-1], epoch)
        self.writer.add_scalar("test_loss", evals_log["test"]["rmse"][-1], epoch)
        

In [8]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {'objective':'reg:squarederror', 'eval_metric': 'rmse', 'learning_rate': 0.1 }
num_boost_round=100

bst = xgb.train(params, dtrain, num_boost_round, evals=[(dtrain, 'train'), (dtest, 'test')],
        callbacks=[TensorBoardCallback()])

[0]	train-rmse:157.20399	test-rmse:147.09992
[1]	train-rmse:143.51689	test-rmse:135.25065
[2]	train-rmse:131.10072	test-rmse:125.39706
[3]	train-rmse:119.98772	test-rmse:117.19328
[4]	train-rmse:109.98798	test-rmse:109.24967
[5]	train-rmse:100.97486	test-rmse:102.32146
[6]	train-rmse:92.80097	test-rmse:96.70532
[7]	train-rmse:85.42460	test-rmse:91.46110
[8]	train-rmse:78.76005	test-rmse:87.08872
[9]	train-rmse:72.79442	test-rmse:83.26352
[10]	train-rmse:67.36035	test-rmse:80.13694
[11]	train-rmse:62.42762	test-rmse:77.12686
[12]	train-rmse:58.08921	test-rmse:74.74316
[13]	train-rmse:54.13500	test-rmse:72.52482
[14]	train-rmse:50.58406	test-rmse:70.60133
[15]	train-rmse:47.24977	test-rmse:69.23445
[16]	train-rmse:44.21664	test-rmse:68.09474
[17]	train-rmse:41.41679	test-rmse:67.10708
[18]	train-rmse:38.87938	test-rmse:66.16503
[19]	train-rmse:36.65492	test-rmse:65.49000
[20]	train-rmse:34.67751	test-rmse:64.94058
[21]	train-rmse:32.88024	test-rmse:64.53994
[22]	train-rmse:31.24105	test-

In [9]:
%tensorboard --logdir ./runs --load_fast=false --bind_all --port 6006